In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
players1 = pd.read_csv("phs_2020_1.csv")
players2 = pd.read_csv("phs_2020_2.csv")
combine = [players1, players2]
players = pd.concat(combine)
players.head(89)

,start_time,esports_match_id,tournament_title,map_type,map_name,player_name,team_name,stat_name,hero_name,stat_amount
0,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,BenBest,Paris Eternal,All Damage Done,All Heroes,6674.164055
1,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,BenBest,Paris Eternal,Assists,All Heroes,11.000000
2,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,BenBest,Paris Eternal,Average Time Alive,All Heroes,60.199502
3,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,BenBest,Paris Eternal,Barrier Damage Done,All Heroes,1875.000000
4,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,BenBest,Paris Eternal,Damage Blocked,All Heroes,14640.911780
...,...,...,...,...,...,...,...,...,...,...
84,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,FDGod,Paris Eternal,Ultimates Earned - Fractional,All Heroes,3.093833
85,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,FDGod,Paris Eternal,Ultimates Used,All Heroes,2.000000
86,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,FDGod,Paris Eternal,Weapon Accuracy,All Heroes,0.263911
87,2/8/2020 18:13,30991,OWL 2020 Regular Season,CONTROL,Lijiang Tower,FDGod,Paris Eternal,All Damage Done,Lúcio,6385.910388


In [10]:
maps = pd.read_csv("match_map_stats.csv")
maps['map_name'].replace({'King\'s Row': 'Kings Row'}, inplace = True)
maps.head(10)

,round_start_time,round_end_time,stage,match_id,game_number,match_winner,map_winner,map_loser,map_name,map_round,...,team_one_name,team_two_name,attacker_payload_distance,defender_payload_distance,attacker_time_banked,defender_time_banked,attacker_control_perecent,defender_control_perecent,attacker_round_end_score,defender_round_end_score
0,2018-01-11 00:12:07,2018-01-11 00:20:07,Overwatch League - Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,1,...,Los Angeles Valiant,San Francisco Shock,75.615051,0.000000,0.000000,240.000000,NaN,NaN,2,0
1,2018-01-11 00:22:05,2018-01-11 00:27:59,Overwatch League - Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,2,...,Los Angeles Valiant,San Francisco Shock,75.649597,75.615051,125.750572,0.000000,NaN,NaN,3,2
2,2018-01-11 00:34:39,2018-01-11 00:38:29,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,1,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,250.492004,240.000000,NaN,NaN,2,0
3,2018-01-11 00:40:27,2018-01-11 00:44:41,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,2,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,225.789032,250.492004,NaN,NaN,2,2
4,2018-01-11 00:46:09,2018-01-11 00:49:48,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,3,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,36.396057,250.492004,NaN,NaN,4,2
5,2018-01-11 00:51:16,2018-01-11 00:56:55,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,4,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,0.000000,36.396057,NaN,NaN,3,4
6,2018-01-11 01:11:32,2018-01-11 01:16:33,Overwatch League - Stage 1,10223,3,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Ilios,1,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,0.000000,0.000000,99.0,100.0,0,1
7,2018-01-11 01:17:19,2018-01-11 01:20:09,Overwatch League - Stage 1,10223,3,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Ilios,2,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,0.000000,0.000000,100.0,0.0,1,1
8,2018-01-11 01:20:55,2018-01-11 01:25:08,Overwatch League - Stage 1,10223,3,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Ilios,3,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,0.000000,0.000000,100.0,65.0,2,1
9,2018-01-11 01:32:26,2018-01-11 01:39:37,Overwatch League - Stage 1,10223,4,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Numbani,1,...,Los Angeles Valiant,San Francisco Shock,75.549507,0.000000,0.000000,0.000000,NaN,NaN,1,0


In [11]:
maps['map_name'].unique()

array(['Dorado', 'Temple of Anubis', 'Ilios', 'Numbani', 'Eichenwalde',
       'Junkertown', 'Oasis', 'Horizon Lunar Colony', 'Lijiang Tower',
       'Volskaya Industries', 'Nepal', 'Kings Row', 'Route 66',
       'Hollywood', 'Hanamura', 'Watchpoint: Gibraltar', 'Blizzard World',
       'Rialto', 'Busan', 'Paris', 'Havana'], dtype=object)

In [12]:
cleaned_maps = pd.DataFrame()
maps.query("`match_id` >= 30991", inplace = True)
maps.query("`match_id` <= 35554", inplace = True)
cleaned_maps['match_id'] = maps['match_id']
cleaned_maps['team_one'] = maps['team_one_name']
cleaned_maps['team_two'] = maps['team_two_name']
cleaned_maps['map_winner'] = maps['map_winner']
cleaned_maps['map_name'] = maps['map_name']
cleaned_maps['game_number'] = maps['game_number']
cleaned_maps['match_winner'] = maps['match_winner']
cleaned_maps.drop_duplicates(inplace = True)
cleaned_maps


,match_id,team_one,team_two,map_winner,map_name,game_number,match_winner
6246,30991,Toronto Defiant,Paris Eternal,Paris Eternal,Lijiang Tower,1,Toronto Defiant
6248,30991,Paris Eternal,Toronto Defiant,Toronto Defiant,Eichenwalde,2,Toronto Defiant
6252,30991,Paris Eternal,Toronto Defiant,Toronto Defiant,Horizon Lunar Colony,3,Toronto Defiant
6255,30991,Paris Eternal,Toronto Defiant,Toronto Defiant,Havana,4,Toronto Defiant
6257,30992,London Spitfire,New York Excelsior,London Spitfire,Lijiang Tower,1,New York Excelsior
...,...,...,...,...,...,...,...
9071,35552,Seoul Dynasty,San Francisco Shock,San Francisco Shock,Kings Row,2,San Francisco Shock
9073,35552,San Francisco Shock,Seoul Dynasty,Seoul Dynasty,Hanamura,3,San Francisco Shock
9075,35552,San Francisco Shock,Seoul Dynasty,Seoul Dynasty,Watchpoint: Gibraltar,4,San Francisco Shock
9077,35552,Seoul Dynasty,San Francisco Shock,San Francisco Shock,Busan,5,San Francisco Shock


In [13]:
new_frame = pd.DataFrame(columns = cleaned_maps.columns)
for index, row in cleaned_maps.iterrows():
    if row['team_two'] < row['team_one']:
        new_frame.loc[index] = [row['match_id'], row['team_two'], row['team_one'], row['match_winner'], row['map_name'], row['game_number'], row['map_winner']]
    else:
        new_frame.loc[index] = [row['match_id'], row['team_one'], row['team_two'], row['match_winner'], row['map_name'], row['game_number'], row['map_winner']]
cleaned_maps = new_frame
cleaned_maps      

,match_id,team_one,team_two,map_winner,map_name,game_number,match_winner
6246,30991,Paris Eternal,Toronto Defiant,Toronto Defiant,Lijiang Tower,1,Paris Eternal
6248,30991,Paris Eternal,Toronto Defiant,Toronto Defiant,Eichenwalde,2,Toronto Defiant
6252,30991,Paris Eternal,Toronto Defiant,Toronto Defiant,Horizon Lunar Colony,3,Toronto Defiant
6255,30991,Paris Eternal,Toronto Defiant,Toronto Defiant,Havana,4,Toronto Defiant
6257,30992,London Spitfire,New York Excelsior,New York Excelsior,Lijiang Tower,1,London Spitfire
...,...,...,...,...,...,...,...
9071,35552,San Francisco Shock,Seoul Dynasty,San Francisco Shock,Kings Row,2,San Francisco Shock
9073,35552,San Francisco Shock,Seoul Dynasty,San Francisco Shock,Hanamura,3,Seoul Dynasty
9075,35552,San Francisco Shock,Seoul Dynasty,San Francisco Shock,Watchpoint: Gibraltar,4,Seoul Dynasty
9077,35552,San Francisco Shock,Seoul Dynasty,San Francisco Shock,Busan,5,San Francisco Shock


In [14]:

win_perc = pd.DataFrame(columns = ['map_name', 'team', 'wins', 'games_played'])
win_dict = {}
played_dict = {}
for val in cleaned_maps['team_one'].unique():
    win_dict[val] = 0
    played_dict[val] = 0
for name in cleaned_maps['map_name'].unique():
    temp_pd = pd.DataFrame()
    team = cleaned_maps.query("`map_name` == '{}'".format(name))
    for index, row in team.iterrows():
        if (row['map_winner'] != 'draw'):
            played_dict[row['team_one']] = played_dict.get(row['team_one'],0) + 1
            played_dict[row['team_two']] = played_dict.get(row['team_two'],0) + 1
            if (row['team_one'] == row['map_winner']):
                win_dict[row['team_one']] = win_dict.get(row['team_one'],0) + 1
            else:
                win_dict[row['team_two']] = win_dict.get(row['team_one'],0) + 1
    for team_name in cleaned_maps['team_one'].unique():
        temp_dict = {'map_name': name, 'team': team_name, 'wins': win_dict.get(team_name, 0), 'games_played': played_dict.get(team_name, 0)}
        win_perc = win_perc.append(temp_dict, ignore_index = True)
    win_dict = {}
    played_dict = {}
    for val in cleaned_maps['team_one'].unique():
        win_dict[val] = 0
        played_dict[val] = 0
win_perc

,map_name,team,wins,games_played
0,Lijiang Tower,Paris Eternal,5,10
1,Lijiang Tower,London Spitfire,3,6
2,Lijiang Tower,Los Angeles Gladiators,6,13
3,Lijiang Tower,Dallas Fuel,4,7
4,Lijiang Tower,Boston Uprising,3,9
...,...,...,...,...
373,Watchpoint: Gibraltar,Toronto Defiant,0,3
374,Watchpoint: Gibraltar,Hangzhou Spark,4,5
375,Watchpoint: Gibraltar,Seoul Dynasty,1,8
376,Watchpoint: Gibraltar,Vancouver Titans,2,5


In [15]:
win_perc['games_played'].replace({0: np.nan}, inplace = True)
win_perc['win_perc'] = win_perc['wins'] / win_perc['games_played']
win_perc

,map_name,team,wins,games_played,win_perc
0,Lijiang Tower,Paris Eternal,5,10.0,0.5
1,Lijiang Tower,London Spitfire,3,6.0,0.5
2,Lijiang Tower,Los Angeles Gladiators,6,13.0,0.461538
3,Lijiang Tower,Dallas Fuel,4,7.0,0.571429
4,Lijiang Tower,Boston Uprising,3,9.0,0.333333
...,...,...,...,...,...
373,Watchpoint: Gibraltar,Toronto Defiant,0,3.0,0.0
374,Watchpoint: Gibraltar,Hangzhou Spark,4,5.0,0.8
375,Watchpoint: Gibraltar,Seoul Dynasty,1,8.0,0.125
376,Watchpoint: Gibraltar,Vancouver Titans,2,5.0,0.4
